# 몬테카를로 학습 (Monte-Carlo Learning)

In [1]:
import numpy as np
import random
from collections import defaultdict
from environment import Env


# 몬테카를로 에이전트 (모든 에피소드 각각의 샘플로 부터 학습)
class MCAgent:                     
    # 생성자: 행동 가치함수 테이블, 파라미터 초기화
    def __init__(self, actions):
        self.width = 5
        self.height = 5
        self.actions = actions
        self.learning_rate = 0.01
        self.discount_factor = 0.9
        self.epsilon = 0.1
        self.samples = []
        self.value_table = defaultdict(float)  #딕셔너리의 디폴트 값은 실수, 키 값은 문자열 '[행, 열]'

    # 에피소드에 샘플을 추가
    def save_sample(self, state, reward, done):
        self.samples.append([state, reward, done])

    # 모든 에피소드에서 에이전트가 방문한 상태의 행동 가치함수(큐 함수)를 업데이트
    def update(self):
        G_t = 0
        visit_state = []
        # 방문 역순으로 에피소드 각 상태의 리턴 게산
        for reward in reversed(self.samples):  
            state = str(reward[0])
            if state not in visit_state:
                visit_state.append(state)
                G_t = reward[1] + self.discount_factor * G_t
                value = self.value_table[state]
                self.value_table[state] = (value +
                                           self.learning_rate * (G_t - value))

    # 입실론 탐욕 정책에 따라 행동을 선택
    def get_action(self, state):
        if np.random.rand() < self.epsilon:
            # 랜덤 행동
            action = np.random.choice(self.actions)
        else:
            # 큐 함수에 따른 행동
            next_state = self.possible_next_state(state)
            action = self.arg_max(next_state)
        return int(action)

    # 모든 행동들의 행동 가치함수(next_state) 중 최대 값의 행동(인덱스) 선택
    @staticmethod
    def arg_max(next_state):
        max_index_list = []
        max_value = next_state[0]
        for index, value in enumerate(next_state):
            if value > max_value:
                max_index_list.clear()
                max_value = value
                max_index_list.append(index)
            elif value == max_value:
                max_index_list.append(index)
        return random.choice(max_index_list)

    # 가능한 다음 모든 상태들의 행동 가치함수 반환
    def possible_next_state(self, state):
        col, row = state
        next_state = [0.0] * 4

        # 위로 이동 다음 상태
        if row != 0:   
            next_state[0] = self.value_table[str([col, row - 1])]
        else:
            next_state[0] = self.value_table[str(state)]
        # 아래로 이동 다음 상태
        if row != self.height - 1:
            next_state[1] = self.value_table[str([col, row + 1])]
        else:
            next_state[1] = self.value_table[str(state)]
        # 왼쪽으로 이동 다음 상태
        if col != 0:
            next_state[2] = self.value_table[str([col - 1, row])]
        else:
            next_state[2] = self.value_table[str(state)]
        # 오른쪽으로 이동 다음 상태
        if col != self.width - 1:
            next_state[3] = self.value_table[str([col + 1, row])]
        else:
            next_state[3] = self.value_table[str(state)]

        return next_state


# 메인 함수
if __name__ == "__main__":
    env = Env()        # 환경 객체 생성
    agent = MCAgent(actions=list(range(env.n_actions)))   # 에이전트 객체 생성

    for episode in range(100):      # 100개의 에피소드에 대해 반복
        state = env.reset()                  # 환경 초기화
        action = agent.get_action(state)     # 현재 상태에서 행동 선택

        while True:           # 한 에피소드 종료될 때까지 반복
            env.render()             # 환경 렌더링

            # 다음 상태로 이동
            # 보상은 숫자이고, 완료 여부는 boolean
            next_state, reward, done = env.step(action)      # 현재 상태에서 행동 후 다음 상태, 보상, 종료 여부 리턴
            agent.save_sample(next_state, reward, done)      # 에피소드의 샘플(상태) 저장

            # 다음 행동 선택
            action = agent.get_action(next_state)

            # 에피소드가 완료됐을 때, 행동 가치함수(큐 함수) 업데이트
            if done:
                agent.update()              # 큐 함수 업데이트
                agent.samples.clear()       # 샘플 초기화
                break